In [1]:
import requests


csv_url = "https://static.openfoodfacts.org/data/en.openfoodfacts.org.products.csv.gz"
response = requests.get(csv_url, stream=True)

if response.status_code == 200:
    with open("../openfoodfacts_products2.csv.gz", "wb") as file:
        for chunk in response.iter_content(chunk_size=8000):
            file.write(chunk)
    print("Télechargement du fichier réussi.")
else:
    print(f"Erreur lors du télechargement du fichier : {response.status_code}")


Télechargement du fichier réussi.


In [1]:
import pyspark
print(pyspark.__version__)

3.5.0


In [2]:
import pyspark.pandas as ps


/usr/local/spark/python/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [3]:

df = ps.read_csv("data/openfoodfacts_products2.csv.gz", sep="\t", mode="PERMISSIVE")

/usr/local/spark/python/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [4]:
nutrition_distinct_list = df['nutriscore_grade'].unique().tolist()

/usr/local/spark/python/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: `to_list` loads all data into the driver's memory. It should only be used if the resulting list is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [11]:
nutrition_distinct_list

['unknown', 'e', 'd', 'c', 'not-applicable', 'b', 'a', None]

In [5]:
df_samples = []
index = 0
for grade in nutrition_distinct_list:
    print(f"{index} - {grade}")
    sample_df = df[df['nutriscore_grade'] == grade].sample(frac=0.1, random_state=42)
    df_samples.append(sample_df)
    index += 1 
df_samples_final = ps.concat(df_samples)



0 - unknown
1 - e
2 - d
3 - c
4 - not-applicable
5 - b
6 - a
7 - None


In [6]:
df_samples_final.to_csv("data/openfoodfacts_products_samples_final.csv.gz", index=False, compression='gzip')

/usr/local/spark/python/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


nettoyage du gros dataset

In [7]:
seuil = 95

colonnes_completement_vides = df.isna().sum() / len(df) * 100 > seuil
colonnes_completement_vides_liste = colonnes_completement_vides[colonnes_completement_vides].index.tolist()
print(len(colonnes_completement_vides_liste))
colonnes_completement_vides_liste

/usr/local/spark/python/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: `to_list` loads all data into the driver's memory. It should only be used if the resulting list is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


116


['abbreviated_product_name',
 'generic_name',
 'packaging_text',
 'origins',
 'origins_tags',
 'origins_en',
 'first_packaging_code_geo',
 'cities',
 'cities_tags',
 'allergens_en',
 'traces',
 'no_nutrition_data',
 'additives',
 'owner',
 'data_quality_errors_tags',
 'energy-from-fat_100g',
 'butyric-acid_100g',
 'caproic-acid_100g',
 'caprylic-acid_100g',
 'capric-acid_100g',
 'lauric-acid_100g',
 'myristic-acid_100g',
 'palmitic-acid_100g',
 'stearic-acid_100g',
 'arachidic-acid_100g',
 'behenic-acid_100g',
 'lignoceric-acid_100g',
 'cerotic-acid_100g',
 'montanic-acid_100g',
 'melissic-acid_100g',
 'unsaturated-fat_100g',
 'monounsaturated-fat_100g',
 'omega-9-fat_100g',
 'polyunsaturated-fat_100g',
 'omega-3-fat_100g',
 'omega-6-fat_100g',
 'alpha-linolenic-acid_100g',
 'eicosapentaenoic-acid_100g',
 'docosahexaenoic-acid_100g',
 'linoleic-acid_100g',
 'arachidonic-acid_100g',
 'gamma-linolenic-acid_100g',
 'dihomo-gamma-linolenic-acid_100g',
 'oleic-acid_100g',
 'elaidic-acid_100

In [8]:
df_sans_colonnes_vides = df.drop(columns=colonnes_completement_vides_liste)

In [9]:
columns_to_be_dropped = ['created_t', 'last_modified_t', 'last_updated_t','quantity', 'packaging', 'packaging_tags', 'packaging_en', 'brands', 'brands_tags', 'brands_en', 'categories_tags', 'categories_en', 'manufacturing_places', 'manufacturing_places_tags', 'labels_en','labels_tags', 'emb_codes','emb_codes_tags', 'purchase_places', 'stores', 'countries','countries_tags', 'countries_en','ingredients_text','ingredients_tags', 'ingredients_analysis_tags', 'allergens', 'traces_tags', 'traces_en', 'serving_size', 'serving_quantity','food_groups_tags' , 'food_groups_en', 'states','states_tags', 'states_en', 'brand_owner', 'environmental_score_score', 'environmental_score_grade','product_quantity', 'unique_scans_n' ,  'popularity_tags', 'completeness',  'last_image_t', 'last_image_datetime', 'main_category_en' , 'image_url', 'image_small_url', 'image_ingredients_url','image_ingredients_small_url', 'image_nutrition_url', 'image_nutrition_small_url' ]

df_sans_colonnes_non_pertinentes = df_sans_colonnes_vides.drop(columns=columns_to_be_dropped)

In [10]:
df_avec_nutriscore = df_sans_colonnes_non_pertinentes[df_sans_colonnes_non_pertinentes['nutriscore_grade'].isin(['a','b', 'c', 'd', 'e'])]

In [12]:
df_avec_nutriscore.to_csv('data/df_avec_nutriscore_clean.csv', index=False)